# `sparse_lap` and `sparse_pflap` quality and speed based on point count

__Goal__: See the relationship between the `sparse_lap` quality and speed based on the number of points used in the algorithm.

__Data__: all `point_counts` * `spacings` * `diag_sizes`

In [1]:
using LAP_julia, TimerOutputs, DataFrames, TableView, JLD2, FileIO, PyPlot, BenchmarkTools, Interact
using LAP_julia: fun_on_dict_values, resize_to_diag_size

┌ Info: Precompiling LAP_julia [797d5b5c-2e9a-429f-9c9f-727808915e33]
└ @ Base loading.jl:1273
Expr(:import, Expr(:., :QuartzImageIO))
  ** incremental compilation may be fatally broken for this module **

:QuartzImageIO = Expr(:ref, Expr(:., :Base, :(:loaded_modules)), Base.PkgId(uuid=Base.UUID(value=UInt128(0xdca85d43d64c5e678c65017450d5d020)), name="QuartzImageIO"))
  ** incremental compilation may be fatally broken for this module **

  likely near /Users/MrTrololord/Google_Drive/cvut/bakalarka/LAP_julia/src/birl.jl:16
  likely near /Users/MrTrololord/Google_Drive/cvut/bakalarka/LAP_julia/src/birl.jl:16
  likely near /Users/MrTrololord/Google_Drive/cvut/bakalarka/LAP_julia/src/birl.jl:16
  likely near /Users/MrTrololord/Google_Drive/cvut/bakalarka/LAP_julia/src/birl.jl:16
  likely near /Users/MrTrololord/Google_Drive/cvut/bakalarka/LAP_julia/src/birl.jl:16
in read_and_release_imgsrc at /Users/MrTrololord/.julia/packages/QuartzImageIO/AjiIs/src/QuartzImageIO.jl
  likely near /Users/

150
LAP_julia succesfully loaded!


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-4017411767714770123\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-672173815618279499\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-4017411767714770123\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-10506825768061520477\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-5852311675925512969\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-10506825768061520477\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
# prepare new dataframe
sp_lap_point_count = DataFrame(
    index = Int[],
    diag_size = Int[],
    whs = Int[],
    timer = TimerOutput[],
    median_results = Dict{String,Float64}[],
    point_count = Int[],
    spacing = Int[],
    points_found = Float16[]
    );

# window half sizes/half sizes of filter bases, also half this number is the maximal displacement.
#window_half_sizes = [15, 30, 45]
window_half_sizes = [25]
# diagonal size of the images
diag_sizes = [400, 600, 900, 1200];
# point counts
point_counts = [50, 200, 350, 500, 650, 800, 1100, 1400]
# minimal number of pixels between two points.
spacings = [5, 15, 35, 50, 60, 80, 100, 120, 140]
# number of anhir images used, these are warped by a known flow.
img_count = 5;

# arrays
errors = Array{Any}(undef, 0);
imgs = Array{Image}(undef, 5);

total_iterations = sum(broadcast(*, length.([window_half_sizes, diag_sizes, point_counts, spacings])...)) * img_count

1440

In [3]:
# prepare images:
for k in 1:img_count
    imgs[k], _ = gen_anhir(mutate=false); # here pass a kwarg: base_path="path/to/anhir/folder/"
end

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = #gen_anhir#29(::Bool, ::Int64, ::typeof(gen_anhir), ::String) at data_gen.jl:10
└ @ LAP_julia /Users/MrTrololord/Google_Drive/cvut/bakalarka/LAP_julia/src/data_gen.jl:10


/Users/MrTrololord/Documents/anhir/dataset/COAD_19/scale-25pc/S3.jpg
sizes: (11214, 11618), (14618, 11399)

  likely near /Users/MrTrololord/.julia/packages/IJulia/DrVMH/src/kernel.jl:52
  likely near /Users/MrTrololord/.julia/packages/IJulia/DrVMH/src/kernel.jl:52
  likely near /Users/MrTrololord/.julia/packages/IJulia/DrVMH/src/kernel.jl:52
  likely near /Users/MrTrololord/.julia/packages/IJulia/DrVMH/src/kernel.jl:52
  likely near /Users/MrTrololord/.julia/packages/IJulia/DrVMH/src/kernel.jl:52
in read_and_release_imgsrc at /Users/MrTrololord/.julia/packages/QuartzImageIO/AjiIs/src/QuartzImageIO.jl
  likely near /Users/MrTrololord/.julia/packages/IJulia/DrVMH/src/kernel.jl:52
in read_and_release_imgsrc at /Users/MrTrololord/.julia/packages/QuartzImageIO/AjiIs/src/QuartzImageIO.jl



/Users/MrTrololord/Documents/anhir/dataset/lung-lobes_3/scale-100pc/29-040-U-35W-Izd1-2-cd31.png
sizes: (11429, 5520), (10583, 5497)
/Users/MrTrololord/Documents/anhir/dataset/lung-lobes_1/scale-100pc/29-039-U-35W-Izd1-3-Pro-SPC.png
sizes: (9776, 5335), (9753, 4731)
/Users/MrTrololord/Documents/anhir/dataset/gastric_6/scale-15pc/CD4.jpg
sizes: (12404, 9860), (12287, 9562)
/Users/MrTrololord/Documents/anhir/dataset/gastric_5/scale-15pc/CD68.jpg
sizes: (12589, 11354), (12928, 11056)


In [4]:
# erase old_results
df = sp_lap_point_count;

## gather data

In [6]:
let index = 0
    for diag_size in diag_sizes
        for whs in window_half_sizes
            for point_count in point_counts, spacing in spacings
                timer = TimerOutput("reg alg: sp lap")
                results_dicts = Array{Dict}(undef, img_count)
                inds_array = Array{Array{Any}}(undef, img_count)

                for k in 1:img_count
                    img = resize_to_diag_size(imgs[k], diag_size)
                    whs_limited = round(Int, min(whs, min(size(img)...)/4))
                    rand_flow = gen_quad_flow(size(img), whs_limited)
                    imgw = warp_img(img, real(rand_flow), imag(rand_flow))
                    try
                        # run registration function
                        _, _, timer, results_dicts[k], (_, inds_array[k]) = test_registration_alg(sparse_lap,
                            img, imgw, rand_flow, [whs_limited], Dict(:timer => timer, :point_count =>
                                point_count, :spacing => spacing), timer=timer, display=false)
                    catch e
                        println(e)
                        push!(errors, [e, img, imgw, rand_flow, whs_limited])
                        continue
                    end
                end
                # if no data gathered:
                if any(x-> 0 .== x, length.([results_dicts, inds_array]))
                    println("SKIPPED: diag_size:", diag_size, " whs: ", whs)
                    continue
                end

                median_results = fun_on_dict_values(filter_defined(results_dicts), median)
                avg_points_found = LAP_julia.mean(map(x -> length(x), filter_defined(inds_array)))

                index = index + 1
                println("at index: ", index, " (of $()", " diag_size: ", diag_size, " whs: ", whs)
                push!(df, Dict(:index => index,
                               :diag_size => diag_size,
                               :whs => whs,
                               :timer => timer,
                               :median_results => median_results,
                               :point_count => point_count,
                               :spacing => spacing,
                               :points_found => avg_points_found
                               ))
            end # point_count in point_counts, spacing in spacings
        end
    end
end

LoadError: syntax: unexpected ")"

## Visualise results

### helper functions

In [5]:
defined_perms(arraylike) = filter(k -> isassigned(arraylike, k), 1:length(arraylike))
filter_defined(arraylike) = arraylike[defined_perms(arraylike)]

filter_defined (generic function with 1 method)